In [1]:
import requests
from bs4 import BeautifulSoup
import ast
import numpy as np
import pandas as pd

In [2]:
headers = {"User-Agent":"Mozilla/5.0"}

page = requests.get('https://www.transfermarkt.it/gianluigi-buffon/profil/spieler/5023', headers=headers)

In [3]:
soup = BeautifulSoup(page.text, 'lxml')

In [5]:
a = 'https://www.transfermarkt.it/gianluigi-buffon/profil/spieler/5023'

In [9]:
a.split('/')[-4]

'gianluigi-buffon'

In [ ]:
soup

In [9]:
soup.find('span', attrs = {'class': 'mediumpunkt'}).text.strip()

'Ligue 1'

In [ ]:
league = soup.find('div', attrs={'class': 'keys'}).find_all('span')[0].text
league

In [ ]:
posizione = soup.find("table", attrs = {"class": "auflistung"}).find_all("tr")[6].find("td").text.strip()
posizione

In [ ]:
righeTab = soup.find("table", attrs = {"class": "auflistung"}).find_all("tr")

In [ ]:
righeTab

In [ ]:
for r in righeTab:
    if "Posizione" in str(r):
        posizione = r.find("td").text.strip()

In [ ]:
posizione

In [ ]:
foot = soup.find("table", attrs = {"class": "auflistung"}).find_all("tr")[7].find("td").text.strip()

In [ ]:
nome = soup.title.text

In [ ]:
nation = soup.find("span",attrs = {"itemprop":"nationality"}).text

In [ ]:
height = soup.find("span",attrs = {"itemprop":"height"}).text

In [ ]:
str_soup = str(soup)

In [ ]:
a = "jQuery(window).on('load',function()" 
b = str_soup.split(a)[1]

In [ ]:
c = b.split("'data'")[1]

In [ ]:
c = c[1:]

In [ ]:
c = c[:(c.find(']')+1)]

In [ ]:
lista = ast.literal_eval(c)

In [ ]:
df = pd.DataFrame(lista)

In [ ]:
df = df.drop(['marker','mw','x'], axis=1)

In [ ]:
df.head()

# FUNZIONE

In [2]:
def ExtractData(url):
    headers = {"User-Agent":"Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'lxml')
        
    righeTab = soup.find("table", attrs = {"class": "auflistung"}).find_all("tr")    
    for r in righeTab:
        if "Posizione" in str(r):
            posizione = r.find("td").text.strip()
        elif "Piede" in str(r):
            foot = r.find("td").text.strip()

    nome = soup.title.text
    league = soup.find('div', attrs={'class': 'keys'}).find_all('span')[0].text
    nation = soup.find("span",attrs = {"itemprop":"nationality"}).text
    age2 = int(soup.find("span",attrs = {"itemprop":"birthDate"}).text.strip()[-3:-1])
    try:
        height = float(soup.find("span",attrs = {"itemprop":"height"}).text[0:4].replace(",","."))
    except:
        height = np.nan
    
    soup = str(soup)
    soup = soup.split("jQuery(window).on('load',function()")[1].split("'data'")[1]
    soup = soup[1:(soup.find(']')+1)]
    lista = ast.literal_eval(soup)
    df = pd.DataFrame(lista).drop(['marker','mw','x'], axis=1)
    df['name'] = nome.split(' - ')[0]
    df['role'] = posizione
    df['league'] = league
    df['nation'] = nation
    df['height'] = height 
    df['foot'] = foot
    df['age2'] = age2
    
    return df

In [ ]:
ExtractData('https://www.transfermarkt.it/franck-ribery/profil/spieler/22068')

In [ ]:
aa='07/apr/1983\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t(35)'

In [ ]:
aa[-3:-1]

In [ ]:
links_leagues = ['https://www.transfermarkt.it/jumplist/startseite/wettbewerb/IT1',
                "https://www.transfermarkt.it/jumplist/startseite/wettbewerb/L1",
                "https://www.transfermarkt.it/jumplist/startseite/wettbewerb/FR1",
                "https://www.transfermarkt.it/jumplist/startseite/wettbewerb/ES1",
                "https://www.transfermarkt.it/jumplist/startseite/wettbewerb/GB1"]

In [ ]:
listone = []

headers = {"User-Agent":"Mozilla/5.0"}
for url in links_leagues:
    index = BeautifulSoup(requests.get(url, headers=headers).text, 'lxml')
    div1 = index.find('div', attrs={'id': 'yw1'})
    table1 = div1.find('table')
    linksTeam = list(set(['https://www.transfermarkt.it'+x['href'] for x in table1.find_all('a', attrs={'class': 'vereinprofil_tooltip'})]))
    for link in linksTeam:
        page = BeautifulSoup(requests.get(link, headers=headers).text, 'lxml')
        table_i = page.find_all('table', attrs={'class': 'items'})[0]
        url_giocatori = list(set(['https://www.transfermarkt.it'+x['href'] for x in table_i.find_all('a', attrs={'class':'spielprofil_tooltip'})]))
        listone += url_giocatori
        

In [ ]:
listone
len(listone)

In [ ]:
df_links_players = pd.DataFrame({'urls': listone})

In [ ]:
df_links_players.head()

In [ ]:
df_links_players.to_csv('urls.csv', index=False)

In [3]:
df_links_players = pd.read_csv('urls.csv')

In [4]:
mydata = pd.DataFrame()
errori_tot = 0

#len(df_links_players.urls)
for i in range(len(df_links_players.urls)):
    try:
        df_help = ExtractData(df_links_players.urls[i])
        mydata = pd.concat([mydata,df_help])
        mydata.reset_index(drop=True, inplace=True)
    except:
        errori_tot+=1
    
    if (i%200)==0:
        print(i, end=' ')
        
print(errori_tot)

0 200 400 600 800 1000 1200 1400 1600 1800 2000 2200 2400 2600 126


In [7]:
mydata.shape

(39918, 11)

In [8]:
mydata.to_csv('mydata.csv', index=False)

In [12]:
sum(mydata.age2 >= 29)

16972